In [ ]:
from numpy import loadtxt
import pandas as pd

#load the data file
data = loadtxt("turbine.dat", skiprows=4)

#convert it to a dataframe using the following fields: height over sea level, fall, net fall, flow, power
df = pd.DataFrame(data, columns=['height', 'fall', 'net_fall', 'flow', 'power'])

#print(df.head)

# Separate the input and output columns
X = df.iloc[:, :-1]  # Select all columns except the last one
y = df.iloc[:, -1]   # Select only the last column

# Split the data into training and testing sets
x_train = X.iloc[:400, :]  # Select the first 400 rows for training
z_train = y.iloc[:400]
x_test = X.iloc[400:, :]   # Select the last 50 rows for testing
z_test = y.iloc[400:]

# print(x_train.shape)
# print(z_train.shape)
# print(x_test.shape)
# print(z_test.shape)
# print(x_train)

In [ ]:
# ======= Data Normalization =======

#store the min and the max of each column to denormalize the data later
min_max = {}
for column in df.columns:
    min_max[column] = [df[column].abs().min(), df[column].abs().max()]

# normalize all the data of the dataframe between 0 and 1
df_scaled = df.copy()

#normalize the data between 0 and 1 using the min_max dictionary
for column in df_scaled.columns:
    df_scaled[column] = 0.1 + 0.8 * (df_scaled[column] - min_max[column][0])  / (min_max[column][1] - min_max[column][0])

# Separate the input and output columns
Xs = df_scaled.iloc[:, :-1]  # Select all columns except the last one
ys = df_scaled.iloc[:, -1]   # Select only the last column

# Split the data into training and testing sets
xs_train = Xs.iloc[:400, :]  # Select the first 400 rows for training
zs_train = ys.iloc[:400]
xs_test = Xs.iloc[400:, :]   # Select the last 50 rows for testing
zs_test = ys.iloc[400:]

print(min_max)

In [ ]:
# ======= Multilayer network with BP =======

import tensorflow as tf

# architecture
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(9, input_dim=4, activation='sigmoid'))
model.add(tf.keras.layers.Dense(3, activation='sigmoid'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

# BP
#sgd = kr.optimizers.SGD(learning_rate=0.1, momentum=0.9)
#sgd = tf.keras.optimizers.legacy.SGD(learning_rate=0.1, momentum=0.9)
sgd = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.fit(xs_train, zs_train, epochs=250, batch_size=1, shuffle=True, verbose=1)

#Store the evolution of the error of the training
model_loss = model.history.history['loss']

# predictions
pred_nn_train = model.predict(xs_train)
pred_nn_test = model.predict(xs_test)

# data denormalization
pred_nn_train = min_max['power'][0] + (pred_nn_train - 0.1) * (min_max['power'][1] - min_max['power'][0]) / 0.8
pred_nn_train = pred_nn_train.transpose()[0]

pred_nn_test = min_max['power'][0] + (pred_nn_test - 0.1) * (min_max['power'][1] - min_max['power'][0]) / 0.8
pred_nn_test = pred_nn_test.transpose()[0]

err_nn_train = 100 * sum(abs(pred_nn_train - z_train)) / sum(z_train)
err_nn_test = 100 * sum(abs(pred_nn_test - z_test)) / sum(z_test)



In [ ]:
#plot the error evolution
import matplotlib.pyplot as plt

plt.plot(model_loss)

#use a logarithmic scale for the y axis
plt.yscale('log')

plt.title('Error evolution')
plt.ylabel('LOSS (MSE)', fontsize=16)
plt.xlabel('# Epoch', fontsize=16)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(8, 8))

# Plot the training set scatter with blue color
ax.scatter(z_train, pred_nn_train, c='blue', label='Training Set')

# Plot the test set scatter with red color
ax.scatter(z_test, pred_nn_test, c='red', label='Test Set')

ax.set_xlabel("real power", fontsize=16)
ax.set_ylabel("predicted power", fontsize=16)

ax.text(0.05, 0.9, "Prediction error TEST (MLR) = %.2f%%\n" % err_nn_train, transform=ax.transAxes, fontsize=16)
ax.text(0.05, 0.85, "Prediction error TRAIN (MLR) = %.2f%%\n" % err_nn_test, transform=ax.transAxes, fontsize=16)

# Add a legend to the plot
ax.legend(loc='lower right', fontsize=14)

#fig.savefig("turbine-mlr-test.png", bbox_inches='tight')
plt.show()
